In [89]:
import pandas as pd
import os
import glob
from functools import reduce
from datetime import date, datetime

In [84]:
stocks = []
companies = []
import glob
for file in glob.glob("./Companies/*"):
    company_name = os.path.basename(file).split(".")[0]
    companies.append(company_name)
    stock = pd.read_csv(file)
    stock = stock[["Date", "Close"]]
    stock.rename(columns={"Close": company_name}, inplace=True)
    stock.set_index(["Date"],inplace=True)
    stocks.append(stock)
    
merged_df = reduce(lambda left,right: pd.merge(left,right, how='outer', left_index=True, right_index=True), stocks)
merged_df = merged_df.drop(labels="2021-11-19", axis=0) # Incomplete data for yesterday

In [104]:
# Verifying nans are only dates before IPO.
for company in companies:
    if "2019-01-02" !=  merged_df[company].first_valid_index():
        nanIndex = merged_df[company][merged_df[company].isnull()].index.tolist()
        print(len(nanIndex))
        start = datetime.strptime(merged_df[company].first_valid_index(), '%Y-%m-%d').date()
        print(start)
        delta = (start - date(2019, 1, 2))
        print(delta.days*5/7)
        print(company, merged_df[company].first_valid_index(), merged_df[company].last_valid_index())

60
2019-03-29
61.42857142857143
LYFT 2019-03-29 2021-11-18
490
2020-12-10
505.7142857142857
ABNB 2020-12-10 2021-11-18
565
2021-03-31
585.0
COUR 2021-03-31 2021-11-18
89
2019-05-10
91.42857142857143
UBER 2019-05-10 2021-11-18
74
2019-04-18
75.71428571428571
ZM 2019-04-18 2021-11-18


In [105]:
merged_df = merged_df.fillna(0)

In [106]:
merged_df.to_csv("./Data/DailyStockData.csv")
